In [1]:
import pandas as pd
import quandl

df = quandl.get("WIKI/GOOGL")

In [6]:
import math
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
import numpy as np

In [4]:
df.head(1)

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0


### Changing df acc. to our own needs

In [7]:
df = df[['Adj. Open',  'Adj. High',  'Adj. Low',  'Adj. Close', 'Adj. Volume']]
df.head(1)

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0


 ### Adding new columns

In [8]:
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Low']) / df['Adj. Close'] * 100.0
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0
df.head(1)

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,HL_PCT,PCT_change
Date,,,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0,8.072956,0.324968


### Defining new df

In [9]:
df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]
df.fillna(-99999, inplace = True )
df.head(2)

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2004-08-19,50.322842,8.072956,0.324968,44659000.0
2004-08-20,54.322689,7.921706,7.227007,22834300.0


In [14]:
df['forecast_col'] = df['Adj. Close']
forecast_out = int(math.ceil(0.01*len(df))) # percent by which to shift 'Adj. Close' upwards
df['label'] = df['forecast_col'].shift(-forecast_out) #shifts upward by value = forecast_out
df.head(2)

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,forecast_col,label
Date,,,,,,
2004-08-19,50.322842,8.072956,0.324968,44659000.0,50.322842,69.078238
2004-08-20,54.322689,7.921706,7.227007,22834300.0,54.322689,67.839414


### Creating X and Y i.e features and labels

In [18]:
df.dropna(inplace = True)
x = np.array(df.drop(['label'],1))
x = preprocessing.scale(x)
y = np.array(df['label'])

[[ 5.03228418e+01  8.07295560e+00  3.24967503e-01  4.46590000e+07
   5.03228418e+01]
 [ 5.43226889e+01  7.92170621e+00  7.22700723e+00  2.28343000e+07
   5.43226889e+01]
 [ 5.48693765e+01  4.04936015e+00 -1.22788010e+00  1.82561000e+07
   5.48693765e+01]
 ...
 [ 1.18159000e+03  1.53098791e+00  4.76194525e-01  2.77496700e+06
   1.18159000e+03]
 [ 1.11920000e+03  1.79860615e+00 -7.29098295e-01  5.79888000e+06
   1.11920000e+03]
 [ 1.06876000e+03  5.45024140e+00 -2.89384977e+00  3.74246900e+06
   1.06876000e+03]]


### Splitting the data into testing and training set

In [16]:
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x,y,test_size = 0.2)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((2711, 5), (678, 5), (2711,), (678,))

### Training with SVR (Support Vector Regression)

In [79]:
clf = svm.SVR()
clf.fit(x_train,y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [81]:
confidence = clf.score(x_test,y_test)
print(confidence)

0.7872718634735898


#### Using different kernels in SVR

In [87]:
for k in ['linear','poly','rbf','sigmoid']:
    clf = svm.SVR(kernel=k)
    clf.fit(x_train, y_train)
    confidence = clf.score(x_test, y_test)
    print(k,confidence)

linear 0.9795914889467047
poly 0.6829424250907499
rbf 0.7872718634735898
sigmoid 0.8998878704893346


### Training with LinearRegression

In [82]:
clf = LinearRegression()
clf.fit(x_train,y_train)
confidence = clf.score(x_test,y_test)
print(confidence)

0.9801365822156849


In [88]:
forecast_out

35

In [19]:
df

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,forecast_col,label
Date,,,,,,
2004-08-19,50.322842,8.072956,0.324968,44659000.0,50.322842,69.078238
2004-08-20,54.322689,7.921706,7.227007,22834300.0,54.322689,67.839414
2004-08-23,54.869377,4.049360,-1.227880,18256100.0,54.869377,68.912727
2004-08-24,52.597363,7.657099,-5.726357,15247300.0,52.597363,70.668146
2004-08-25,53.164113,3.886792,1.183658,9188600.0,53.164113,71.219849
2004-08-26,54.122070,3.048837,2.820391,7094800.0,54.122070,72.278116
2004-08-27,53.239345,2.760245,-1.803885,6211700.0,53.239345,74.810934
2004-08-30,51.162935,3.411430,-3.106003,5196700.0,51.162935,74.199045
2004-08-31,51.343492,1.514115,0.048866,4917800.0,51.343492,70.462511
